In [1]:
from mlutils import dataset
from mlutils import connector

drop_daas_table_df = "drop table wmt-tns-analytics-prod.TnS_adhoc.LMD_DAAS"
print(drop_daas_table_df)

print("drop daas table query executin")

drop_daas_table_df = dataset.load(name="bq_connection", query=drop_daas_table_df)

print("drop daas table query executed")

create_daas_table_df = "create table `wmt-tns-analytics-prod.TnS_adhoc.LMD_DAAS` as \
( \
select \
TENANT_ID, TENANT_NM, WM_WK,DLVR_AREA_NM,ST_CD,EXT_STORE_NBR, \
count (distinct sales_order_nbr) as Total_Orders, \
count (distinct case when batch_ind=1 then sales_order_nbr end) as totalBatch_Orders, \
count (distinct case when TEMPRATURE_CNTRL_ITEM_IND = 1 then sales_order_nbr end) as ColdChainOrders, \
count (distinct case when ORDER_COLD_CHN_CMPLY_IND = 1 then sales_order_nbr end) as ColdChainExceptionOrders, \
count (distinct case when TEMPRATURE_CNTRL_ITEM_IND = 1 and DLVR_STS = 'RETURNED' then sales_order_nbr end) as ColdChain_RTNOrders, \
count (distinct case when TEMPRATURE_CNTRL_ITEM_IND = 1 and DLVR_STS = 'RETURNED' and CLNT_EXCPT_RSN_CD in ('RT-004')then sales_order_nbr end) as ColdChain_RTN004, \
count (distinct case when TEMPRATURE_CNTRL_ITEM_IND = 1 and DLVR_STS = 'CANCELLED' then sales_order_nbr end) as ColdChain_CNCL, \
count (distinct case when TEMPRATURE_CNTRL_ITEM_IND = 1 and DLVR_STS = 'DELIVERED' then sales_order_nbr end) as ColdChain_DelOrders, \
count (distinct case when TEMPRATURE_CNTRL_ITEM_IND = 1 and batch_ind=1 then sales_order_nbr end) as ColdChainBatch_Orders, \
count (distinct case when TEMPRATURE_CNTRL_ITEM_IND = 1 and batch_ind=1 and DLVR_STS = 'RETURNED' then sales_order_nbr end) as ColdChainBatch_RTN, \
count (distinct case when TEMPRATURE_CNTRL_ITEM_IND = 1 and batch_ind=1 and DLVR_STS = 'RETURNED' and CLNT_EXCPT_RSN_CD in ('RT-004') then sales_order_nbr end) as ColdChainBatch_RTN004, \
count (distinct case when TEMPRATURE_CNTRL_ITEM_IND = 1 and batch_ind=1 and DLVR_STS = 'CANCELLED' then sales_order_nbr end) as ColdChainBatch_CNCL, \
count (distinct case when TEMPRATURE_CNTRL_ITEM_IND = 1 and batch_ind=1 and DLVR_STS = 'DELIVERED' then sales_order_nbr end) as ColdChainBatch_Del, \
SUM(case when BATCH_SIZE = TRIP_DLVR_SEQ_NBR then DATETIME_DIFF(coalesce(DLVR_CARRIER_TS, RTN_TS), ENRTE_TO_PCKUP_TS, hour) else 0 end) AS TOT_TRIP_LAST_MI_TM_CNT \
from `wmt-edw-sandbox.LM_IAD_DAAS.DPS_DLMD_ALL` \
where wm_wk > '202201' \
group by 1,2,3,4,5,6 \
) "

print(create_daas_table_df)

print("create table query executing")

create_daas_table_df = dataset.load(name="bq_connection", query=create_daas_table_df)

drop table wmt-tns-analytics-prod.TnS_adhoc.LMD_DAAS
drop daas table query executin
drop daas table query executed
create table `wmt-tns-analytics-prod.TnS_adhoc.LMD_DAAS` as ( select TENANT_ID, TENANT_NM, WM_WK,DLVR_AREA_NM,ST_CD,EXT_STORE_NBR, count (distinct sales_order_nbr) as Total_Orders, count (distinct case when batch_ind=1 then sales_order_nbr end) as totalBatch_Orders, count (distinct case when TEMPRATURE_CNTRL_ITEM_IND = 1 then sales_order_nbr end) as ColdChainOrders, count (distinct case when ORDER_COLD_CHN_CMPLY_IND = 1 then sales_order_nbr end) as ColdChainExceptionOrders, count (distinct case when TEMPRATURE_CNTRL_ITEM_IND = 1 and DLVR_STS = 'RETURNED' then sales_order_nbr end) as ColdChain_RTNOrders, count (distinct case when TEMPRATURE_CNTRL_ITEM_IND = 1 and DLVR_STS = 'RETURNED' and CLNT_EXCPT_RSN_CD in ('RT-004')then sales_order_nbr end) as ColdChain_RTN004, count (distinct case when TEMPRATURE_CNTRL_ITEM_IND = 1 and DLVR_STS = 'CANCELLED' then sales_order_nbr end) a

In [ ]:
from mlutils import dataset
from mlutils import connector

create_coldchain_table_df =  "create table `wmt-tns-analytics-prod.TnS_adhoc.ColdChain_Data` as \
    ( \
select t1.cold_chain_time, t2.* from \
(select PO_NUM, max(SRC_MODFD_TS) as cold_chain_time from `wmt-edw-prod.WW_SUPPLY_CHAIN_DL_VM.LAST_MI_DLVR_TRK_STATUS` \
where src_cre_dt >= '2023-02-01'  and (CARRIER_STS_CD = 'COLD_CHAIN_COMPLIANCE_INITIATED') and PO_NUM in ( \
select distinct PO_NUM from  `wmt-edw-prod.WW_SUPPLY_CHAIN_DL_VM.LAST_MI_DLVR_TRK_STATUS` \
where src_cre_dt >= '2023-02-01' and CARRIER_STS_CD = 'COLD_CHAIN_COMPLIANCE_INITIATED' \
and carrier_nm = 'IN_HOME_ASSOCIATE') \
GROUP BY PO_NUM) t1 \
left join( \
select  \
  PO_NUM, \
  TRIP_ID, \
  DLVR_REF_ID, \
  SRC_SALES_ORDER_NUM, \
  CARRIER_STS_CD, \
  MAX (CASE WHEN CARRIER_STS_CD = 'DL' THEN SRC_CRE_TS_UTC END) as DLR_TS, \
  MAX (CASE WHEN CARRIER_STS_CD = 'CC' THEN SRC_CRE_TS_UTC END) as RTS_TS, \
  MAX (CASE WHEN CARRIER_STS_CD = 'RT' THEN SRC_CRE_TS_UTC END) as CNL_TS, \
   FROM `wmt-edw-prod.WW_SUPPLY_CHAIN_DL_SECURE.LAST_MI_DLVR_TRK_STATUS` LMDTS \
  WHERE LMDTS.CARRIER_NM = 'IN_HOME_ASSOCIATE' /* InHome driver. */ \
       AND LMDTS.TRIP_SVC_TYPE_NM = 'DELIVERY' /* Trip for deliveries. */ \
       AND LMDTS.SRC_CRE_DT > '2023-02-01' /* Random date filter. */ \
    GROUP BY 1,2,3,4,5 \
) t2 \
on t1.PO_NUM =t1.PO_NUM \
limit 10000) "


print("create table query executing")

create_coldchain_table_df = dataset.load(name="bq_connection", query=create_coldchain_table_df)